In [ ]:
# Import necessary packages
import pandas as pd
from omegaconf import OmegaConf

In [ ]:
# Global configs
yaml_cfg = """
seed: 42
"""
config = OmegaConf.create(yaml_cfg)
print(config.seed)


In [ ]:
# Read raw data
df_train: pd.DataFrame = pd.read_excel('./raw_data/time_series_375_prerpocess_en.xlsx')
print(df_train.describe())

Steps:

- fill `patient_id` and only reserve y-m-d for `RE_DATE` column
- merge lab tests in the same date
- normalize data
- fill missing data (several options)
- feature selection
- combine above data to time series data (one patient one record)
- export to python pickle file

In [ ]:
df_train['PATIENT_ID'].fillna(method='ffill', inplace=True)
df_train['RE_DATE'] = df_train['RE_DATE'].dt.strftime('%Y-%m-%d')
print(df_train)

In [ ]:
# For the same patient and the same date, merge multiple lab tests into one data frame

def merge_lab_test(df: pd.DataFrame) -> pd.DataFrame:
    dates = pd.to_datetime(df_train['RE_DATE'], format='%Y%m%d')
    print(dates.dt.strftime('%Y-%m-%d'))
    return df

df = merge_lab_test(df_train)
print(df['date'])